## 初始demo


### cpu并行设置

In [ ]:
! ipcluster start -n 24

In [ ]:
! ipcluster stop

In [ ]:
import ipyparallel as ipp

# 首先，您需要启动一个集群（在笔记本的“Clusters”选项卡中）或使用命令行（通过 'ipcluster start -n 4'，4 是您想要的核心数）

# 创建一个客户端实例
client = ipp.Client()

# 每个“引擎”都是一个单独的Python实例
dview = client[:]

# 定义我们想要并行运行的函数
def my_function(x):
    return x * x

# 用'map'方法并行运行函数
result = dview.map_sync(my_function, range(8))

print(result)

client.close()

### QEC 测试

In [1]:
%run qsu.ipynb  # color-printing functions
import numpy as np
from qecsim import paulitools as pt
from qecsim.models.generic import DepolarizingErrorModel, BitFlipErrorModel, PhaseFlipErrorModel
from qecsim.models.planar import PlanarCode, PlanarMWPMDecoder, PlanarMPSDecoder
from ti_decoder import ti_calc_serial, ti_calc_Parallel, ti_calc_serial_spin_encoded

# initialise models
d = 7
my_code = PlanarCode(d, d)
my_error_model = PhaseFlipErrorModel()
#my_decoder = PlanarMWPMDecoder()
# print models
print(my_code)
print(my_error_model)
#print(my_decoder)

PlanarCode(7, 7)
PhaseFlipErrorModel()


In [2]:
# set physical error probability to 10%
error_probability = 0.1
# seed random number generator for repeatability
rng = np.random.default_rng(1111)

# error: random error based on error probability
error = my_error_model.generate(my_code, error_probability, rng)
qsu.print_pauli('error:\n{}'.format(my_code.new_pauli(error)))

In [4]:
# syndrome: stabilizers that do not commute with the error
syndrome = pt.bsp(error, my_code.stabilizers.T)
qsu.print_pauli('syndrome:\n{}'.format(my_code.ascii_art(syndrome)))

### 自定义MPS Decoder计算coset概率

Please add the following code to PlanarMPSDecoder class
```python
def calc_coset_probabilities(self, code, syndrome,
            error_model=DepolarizingErrorModel(),  # noqa: B008
            error_probability=0.1, **kwargs):
    """
    See :meth:`qecsim.model.Decoder.decode`

    Note: The optional keyword parameters ``error_model`` and ``error_probability`` are used to determine the prior
    probability distribution for use in the decoding algorithm. Any provided error model must implement
    :meth:`~qecsim.model.ErrorModel.probability_distribution`.

    :param code: Planar code.
    :type code: PlanarCode
    :param syndrome: Syndrome as binary vector.
    :type syndrome: numpy.array (1d)
    :param error_model: Error model. (default=DepolarizingErrorModel())
    :type error_model: ErrorModel
    :param error_probability: Overall probability of an error on a single qubit. (default=0.1)
    :type error_probability: float
    :return: Recovery operation as binary symplectic vector.
    :rtype: numpy.array (1d)
    """
    # any recovery
    any_recovery = self.sample_recovery(code, syndrome)
    # probability distribution
    prob_dist = error_model.probability_distribution(error_probability)
    # coset probabilities, recovery operations
    coset_ps, recoveries = self._coset_probabilities(prob_dist, any_recovery)

    return coset_ps, recoveries
```


### 生成Recovery

In [5]:
# from mps decoder
my_mps_decoder = PlanarMPSDecoder(chi=6)
coset_ps, recoveries = my_mps_decoder.calc_coset_probabilities(
    my_code,
    syndrome,
    error_model=my_error_model,
    error_probability=error_probability)

print('total prob of each coset', coset_ps)
print('Only the first and last are non-zero because only Z errors.')
print('The log of their ratio is:',
      np.log(float(coset_ps[3]) / float(coset_ps[0])))


total prob of each coset (mpf('1.5758944953105232e-11'), mpf('0.0'), mpf('0.0'), mpf('2.6072957444614728e-12'))
Only the first and last are non-zero because only Z errors.
The log of their ratio is: -1.7990945666238425


In [6]:
recovery = recoveries[0].to_bsf()
print(recovery)
qsu.print_pauli('recovery:\n{}'.format(my_code.new_pauli(recovery)))

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 1
 0 0 0 0 0 0 1 1 1 1 0 1 0 1 1 1 1 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [7]:
# check recovery ^ error commutes with stabilizers (by construction)
print(pt.bsp(recovery ^ error, my_code.stabilizers.T))

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0]


In [8]:
# success iff recovery ^ error commutes with logicals
print(pt.bsp(recovery ^ error, my_code.logicals.T))

[0 0]


In [9]:
# couriosity:recovery ^ error
qsu.print_pauli('recovery ^ error:\n{}'.format(my_code.new_pauli(recovery ^ error)))

Note: The decoder is not guaranteed to find a successful recovery operation. The planar $5 \times 5$ code has distance $d=5$ so we can only guarantee to correct errors up to weight $(d-1) / 2=2$.

### TI程序并行化测试

In [ ]:
ti_calc_serial(recovery,
           error_probability=error_probability,
           d=d,
           num_int_steps=101,
           mcmove_steps_for_derivative=100, 
           num_samples=1000)

In [10]:
print('d={:}'.format(d))
ti_calc_Parallel(recovery,
           error_probability=error_probability,
           d=d,
           num_int_steps=101,
           mcmove_steps_init = 500000, 
           mcmove_steps_for_derivative=1000, 
           num_samples=1000)
#print(f"d={d}")

d=7
0.056
0.057
0.057
0.058
0.058
0.059
0.059
0.059
0.059
0.060
0.060
0.060
0.061
0.060
0.061
0.062
0.062
0.062
0.064
0.064
0.064
0.064
0.065
0.066
0.066
0.067
0.067
0.067
0.068
0.069
0.069
0.069
0.070
0.070
0.071
0.071
0.072
0.071
0.073
0.073
0.073
0.073
0.074
0.074
0.074
0.074
0.075
0.076
0.076
0.077
0.078
0.076
0.078
0.077
0.076
0.076
0.075
0.075
0.075
0.075
0.075
0.075
0.074
0.074
0.074
0.074
0.073
0.074
0.074
0.074
0.074
0.074
0.073
0.074
0.074
0.074
0.075
0.075
0.075
0.075
0.076
0.075
0.075
0.075
0.075
0.076
0.076
0.076
0.075
0.075
0.076
0.076
0.076
0.076
0.075
0.076
0.076
0.076
0.076
0.075
0.074


(-1.7879036108242559, 0.01740242589720967)

### TI 自旋编码测试

In [ ]:
ti_calc_serial_spin_encoded(recovery,
           error_probability=error_probability,
           d=d,
           num_int_steps=101,
           mcmove_steps_for_derivative=100, 
           num_samples=1000)

### cpu并行TI测试

#### 串行TI,不做error bar

In [ ]:
import ipyparallel as ipp
import time

# 创建一个客户端实例来连接集群
client = ipp.Client()

# 获取所有可用的引擎
dview = client[:]

# 导入您的函数
from ti_decoder import ti_calc

# 这是固定的参数
recovery = recovery
error_probability = error_probability
d = d
print(f"d={d}, error_probability={error_probability}")
# print recovery ^ error (out of curiosity)查看距离和纠错
qsu.print_pauli('recovery ^ error:\n{}'.format(my_code.new_pauli(recovery ^ error)))

# 将函数和变量推送到所有的引擎
dview.push({'ti_calc': ti_calc, 'recovery': recovery, 'error_probability': error_probability, 'd': d})

# 这些是变化的参数
num_int_steps_values = list(range(401, 801, 50))
mcmove_steps_values = list(range(100, 201, 20))  # 从 ... 到 ...，步长为 ...

# 创建一个参数列表，其中每个元素都是一个 (num_int_steps, mcmove_steps) 对
params = [(num_steps, mc_steps) for num_steps in num_int_steps_values for mc_steps in mcmove_steps_values]

# 定义并行函数，它将在每个引擎上被调用
def parallel_function(args):
    num_int_steps, mcmove_steps_for_derivative = args  # 从参数对中解包值
    return ti_calc(recovery, error_probability, d, num_int_steps=num_int_steps, mcmove_steps_for_derivative=mcmove_steps_for_derivative)

# 使用 'map' 方法并行运行函数
#results = dview.map_sync(parallel_function, params)

# 使用 'map_async' 而不是 'map_sync' 来开始您的计算，这样它们就不会阻塞
async_result = dview.map_async(parallel_function, params)

print("计算开始...")

while not async_result.ready():
    print(f"完成度: {100 * (1 - len(client.outstanding) / len(async_result.msg_ids)):.2f}%")
    time.sleep(15)  # 每5秒检查一次

print("计算完成。")

results = async_result.get()

# 对于每个参数对和对应的结果，打印输出
for param, result in zip(params, results):
    num_steps, mc_steps = param
    print(f"num_int_steps: {num_steps}, mcmove_steps: {mc_steps}, result: {result}\n")  

client.close()


#### 并行TI sample,做error bar

In [ ]:
import ipyparallel as ipp
import time

# 创建一个客户端实例来连接集群
client = ipp.Client()

# 获取所有可用的引擎
dview = client[:]

# 导入您的函数
from ti_decoder import ti_calc_Parallel

# 这是固定的参数
recovery = recovery
error_probability = error_probability
d = d
mcmove_steps_for_derivative = 60000
num_samples = 1000
print(f"d={d}, error_probability={error_probability}")

# 将函数和变量推送到所有的引擎
dview.push({
    'ti_calc_Parallel': ti_calc_Parallel, 
    'recovery': recovery, 
    'error_probability': error_probability, 
    'd': d, 
    'mcmove_steps_for_derivative': mcmove_steps_for_derivative, 
    'num_samples': num_samples
})

# 这些是变化的参数
num_int_steps_values = list(range(101, 562, 20))

# 定义并行函数，它将在每个引擎上被调用
def parallel_function(num_int_steps):
    return ti_calc_Parallel(recovery, error_probability, d, num_int_steps=num_int_steps, mcmove_steps_for_derivative=mcmove_steps_for_derivative, num_samples=num_samples)

# 使用 'map' 方法并行运行函数
async_result = dview.map_async(parallel_function, num_int_steps_values)

print("计算开始...")

while not async_result.ready():
    print(f"完成度: {100 * (1 - len(client.outstanding) / len(async_result.msg_ids)):.2f}%")
    time.sleep(15)  # 每15秒检查一次

print("计算完成。")

results = async_result.get()

# 对于每个参数和对应的结果，打印输出
for num_steps, result in zip(num_int_steps_values, results):
    mean_integral, error_bar = result
    print(f"num_int_steps: {num_steps}, mean integral: {mean_integral}, error bar: {error_bar}\n")  

# 使用with语句和open函数打开一个新的文本文件（如果文件不存在，它会被创建）
with open("output.txt", "w") as file:
    file.write(f"d={d}, error_probability={error_probability}, mcmove_steps = {mcmove_steps_for_derivative}, num_samples = {num_samples}\n")
    file.write(f"The log of their ratio is: {np.log(float(coset_ps[3]) / float(coset_ps[0]))}\n\n")
    for num_steps, result in zip(num_int_steps_values, results):
        mean_integral, error_bar = result
        file.write(f"num_int_steps: {num_steps}, mean integral: {mean_integral}, error bar: {error_bar}\n")

client.close()

## Try starting configuration from MWPM

In [ ]:
mwpm = PlanarMWPMDecoder()
mwpm_recovery = mwpm.decode(my_code, syndrome)
qsu.print_pauli('recovery:\n{}'.format(my_code.new_pauli(mwpm_recovery)))

In [ ]:
# print recovery ^ error (out of curiosity)
qsu.print_pauli('recovery ^ error:\n{}'.format(my_code.new_pauli(mwpm_recovery ^ error)))

In [ ]:
# success iff recovery ^ error commutes with logicals
print(pt.bsp(mwpm_recovery ^ error, my_code.logicals.T))

In [ ]:
ti_calc(mwpm_recovery, error_probability=error_probability,
        d=d,
        num_int_steps=101)
